In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
from pandas.tseries.offsets import Hour
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from keras.preprocessing.sequence import TimeseriesGenerator
import xgboost as xgb

np.random.seed(42)


In [2]:

global busses
busses =        ['310_PV_1', '310_PV_2', '320_PV_1', '314_PV_1', '314_PV_2', '313_PV_1', '314_PV_3', '314_PV_4', '313_PV_2', '324_PV_1',    
                 '312_PV_1', '324_PV_2', '324_PV_3', '113_PV_1', '319_PV_1', '215_PV_1', '102_PV_1', '101_PV_1', '102_PV_2',
                 '104_PV_1', '101_PV_2', '101_PV_3', '101_PV_4', '103_PV_1', '119_PV_1']



In [3]:
df = pd.read_csv('PV_deseason_15.csv', parse_dates=[0], index_col=0)

In [4]:
def lags(df, n):
    
    df_new = df
    
    for i in range(n):
        name = "lag "+ str(i+1)
        df_new[name] = df['310_PV_1'].shift(i+1)
    
    return df_new
    

In [5]:
def TrainTest(df):
    
    num_rows = df.shape[0]
    
    num_training = (num_rows * 8)//10
   
    #train = df.iloc[0:num_training]
    #test = df.iloc[num_training:num_rows]
    
    
    return num_training 

In [21]:
def TestModel(X,test, mod):
    
    MSEs = []
    
    for i in range(len(X)):
        
        print(X.iloc[i].to_frame().transpose())
        pred = mod.predict(X.iloc[i].to_frame().transpose())
        #pred = scaler.inverse_transform(pred)
        
        print(pred)
        print(test.iloc[i])
        
        MSEs.append(mse([test.iloc[i]],pred))
    
        
        
    return MSEs

In [22]:



def model1Train(X_train,Y_train,X_test,Y_test):
    mod = xgb.XGBRegressor()
    mod.fit(X_train, Y_train, eval_set = [(X_train, Y_train),(X_test, Y_test)])#keep low for now, edit later eval_set = [(X_train, y_train),(X_test, y_test)]
    
    return mod

    

In [23]:
def model2Train(X_train,Y_train):
    mod = xgb.XGBRegressor()
    mod.fit(X_train, Y_train)#keep low for now, edit later eval_set = [(X_train, y_train),(X_test, y_test)]
    
    return mod

In [24]:
def runModels(df, last):
    scaler = MinMaxScaler()
     
    bound = TrainTest(df)
    
    
    last_index =pd.DatetimeIndex(data = [idx.replace(year = 2019) for idx in last.index])
    df_index = df.index
    new_df_idx = last_index.append(df_index)
    
    array_df = df['310_PV_1'].to_numpy()
    
    array_df = np.append(last['310_PV_1'].to_numpy(), array_df)
    
    
    
    new_df = pd.DataFrame(array_df, columns = ['310_PV_1'])
    new_df['Dates'] = new_df_idx
    new_df = new_df.set_index('Dates')
    

    
    
    lag = lags(new_df, 107)
    
    lag = lag[lag.columns[::-1]]
    

    
    
    lagged_train = lag.iloc[107:bound+107]
    lagged_test = lag.iloc[bound+107:]

    X_train = lagged_train.drop('310_PV_1', axis = 1)
    Y_train = lagged_train['310_PV_1']
    
    X_test = lagged_test.drop('310_PV_1', axis = 1)
    Y_test = lagged_test['310_PV_1']
    
    print(Y_train[1])
    print(Y_test[1])
  
    
    #mod = model1Train(X_train, Y_train,X_test, Y_test)
    mod = model2Train(X_train, Y_train)
    
    Tests = TestModel(X_test, Y_test, mod)
    
    print(Tests)
    
    return Tests

    

In [25]:

df1 = df[df.index.month.isin([1,2,3])]
df1 = df1.iloc[:-1 , :]
print(df1)
df2 = df[df.index.month.isin([4,5,6])]
df3 = df[df.index.month.isin([7,8,9])]
df4 = df[df.index.month.isin([10,11,12])]

                     320_PV_1  314_PV_1  314_PV_2  313_PV_1  314_PV_3  \
Date                                                                    
2020-01-01 00:15:00  3.966453  3.980737  4.282982  7.276192  6.895283   
2020-01-01 00:30:00  3.966107  3.980367  4.282545  7.275664  6.894732   
2020-01-01 00:45:00  3.965761  3.979998  4.282108  7.275137  6.894181   
2020-01-01 01:00:00  3.965415  3.979628  4.281671  7.274609  6.893630   
2020-01-01 01:15:00  3.965069  3.979259  4.281233  7.274081  6.893079   
...                       ...       ...       ...       ...       ...   
2020-03-31 22:45:00  0.880274  0.763739  0.579910  2.526283  2.095675   
2020-03-31 23:00:00  0.879928  0.763376  0.579482  2.525721  2.095125   
2020-03-31 23:15:00  0.879581  0.763014  0.579053  2.525159  2.094575   
2020-03-31 23:30:00  0.879234  0.762651  0.578625  2.524598  2.094024   
2020-03-31 23:45:00  0.878887  0.762289  0.578196  2.524036  2.093474   

                     314_PV_4  313_PV_2  310_PV_1 

In [26]:
mse1 = runModels(df1,df4.iloc[-107:])

C:\Users\bvits\AppData\Local\Temp\ipykernel_11788\239345827.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[name] = df['310_PV_1'].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_11788\239345827.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new[name] = df['310_PV_1'].shift(i+1)
C:\Users\bvits\AppData\Local\Temp\ipykernel_11788\239345827.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

4.125291466810914
1.6454898259911186
                     lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-13 19:15:00 -2.61448 -1.724143  4.739966 -6.519696 -6.418702   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-13 19:15:00 -4.580824 -2.443491 -0.402061  1.296474  1.654844  ...   

                        lag 10      lag 9     lag 8     lag 7     lag 6  \
2020-03-13 19:15:00 -15.331883 -12.029519 -5.359668 -6.317959 -4.866415   

                        lag 5     lag 4     lag 3     lag 2     lag 1  
2020-03-13 19:15:00 -2.831537 -0.714418  0.902427  1.503425  1.641035  

[1 rows x 107 columns]
[-0.9434131]
1.6458579469421952
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-13 19:30:00 -1.724143  4.739966 -6.519696 -6.418702 -4.580824   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-13 19:30:00 -2.443491 -0.402061  1.296474  1.654844  1.678515  ...   

      

                      lag 107   lag 106   lag 105   lag 104  lag 103  \
2020-03-13 22:45:00  1.677571  1.677198  1.676826  1.676453  1.67608   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-13 22:45:00  1.675707  1.675334  1.674961  1.674588  1.674215  ...   

                       lag 10     lag 9     lag 8     lag 7     lag 6  \
2020-03-13 22:45:00  1.644385  1.644017  1.643649  1.643281  1.642912   

                        lag 5     lag 4     lag 3     lag 2     lag 1  
2020-03-13 22:45:00  1.642544  1.642176  1.641807  1.641439  1.641071  

[1 rows x 107 columns]
[1.6410704]
1.6407023998955133
                      lag 107   lag 106   lag 105  lag 104   lag 103  \
2020-03-13 23:00:00  1.677198  1.676826  1.676453  1.67608  1.675707   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-13 23:00:00  1.675334  1.674961  1.674588  1.674215  1.673842  ...   

                       lag 10     lag 9     lag 8 

                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-14 02:15:00  1.672351  1.671978  1.671605  1.673778  1.673411   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-14 02:15:00  1.673044  1.672678  1.672311  1.671944  1.671578  ...   

                       lag 10     lag 9     lag 8     lag 7    lag 6  \
2020-03-14 02:15:00  1.639226  1.638857  1.638488  1.638119  1.63775   

                        lag 5     lag 4     lag 3     lag 2     lag 1  
2020-03-14 02:15:00  1.637381  1.637012  1.636643  1.636274  1.635905  

[1 rows x 107 columns]
[1.6359044]
1.6355357240606505
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-14 02:30:00  1.671978  1.671605  1.673778  1.673411  1.673044   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-14 02:30:00  1.672678  1.672311  1.671944  1.671578  1.671211  ...   

                       lag 10     lag 9     lag 

                      lag 107   lag 106  lag 105   lag 104   lag 103  \
2020-03-14 05:45:00  1.669744  1.669377  1.66901  1.668643  1.668276   

                      lag 102   lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-14 05:45:00  1.667909  1.667542  1.667175  1.666808  1.432834  ...   

                       lag 10     lag 9     lag 8     lag 7     lag 6  \
2020-03-14 05:45:00  1.634059  1.633691  1.633324  1.632956  1.632588   

                       lag 5     lag 4     lag 3     lag 2     lag 1  
2020-03-14 05:45:00  1.63222  1.631852  1.631484  1.359258  0.233207  

[1 rows x 107 columns]
[0.23320739]
-0.813062343570877
                      lag 107  lag 106   lag 105   lag 104   lag 103  \
2020-03-14 06:00:00  1.669377  1.66901  1.668643  1.668276  1.667909   

                      lag 102   lag 101   lag 100    lag 99   lag 98  ...  \
2020-03-14 06:00:00  1.667542  1.667175  1.666808  1.432834  0.33438  ...   

                       lag 10     lag 9     lag 8    

...

[1.4980876]
1.497720009458968
                      lag 107   lag 106   lag 105  lag 104   lag 103  \
2020-03-18 01:45:00  1.536742  1.536375  1.536007  1.53564  1.535272   

                      lag 102   lag 101  lag 100    lag 99    lag 98  ...  \
2020-03-18 01:45:00  1.534905  1.534537  1.53417  1.533802  1.533434  ...   

                       lag 10     lag 9     lag 8     lag 7    lag 6  \
2020-03-18 01:45:00  1.501031  1.500663  1.500296  1.499928  1.49956   

                        lag 5     lag 4     lag 3     lag 2    lag 1  
2020-03-18 01:45:00  1.499192  1.498824  1.498456  1.498088  1.49772  

[1 rows x 107 columns]
[1.4977196]
1.4973520738723654
                      lag 107   lag 106  lag 105   lag 104   lag 103  \
2020-03-18 02:00:00  1.536375  1.536007  1.53564  1.535272  1.534905   

                      lag 102  lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-18 02:00:00  1.534537  1.53417  1.533802  1.533434  1.533067  ...   

                       lag 1

[1.4933019]
1.4929338469004865
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-18 05:00:00  1.531964  1.531596  1.531229  1.530861  1.530493   

                      lag 102   lag 101  lag 100    lag 99    lag 98  ...  \
2020-03-18 05:00:00  1.530126  1.529758  1.52939  1.529022  1.528654  ...   

                       lag 10    lag 9     lag 8     lag 7     lag 6  \
2020-03-18 05:00:00  1.496248  1.49588  1.495511  1.495143  1.494775   

                        lag 5     lag 4     lag 3     lag 2     lag 1  
2020-03-18 05:00:00  1.494407  1.494039  1.493671  1.493302  1.492934  

[1 rows x 107 columns]
[1.4929335]
1.4925654832685886
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-18 05:15:00  1.531596  1.531229  1.530861  1.530493  1.530126   

                      lag 102  lag 101   lag 100    lag 99    lag 98  ...  \
2020-03-18 05:15:00  1.529758  1.52939  1.529022  1.528654  1.528286  ...   

                     

                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-18 08:15:00  0.129641 -0.916628 -3.067816 -5.736217 -8.008443   

                      lag 102   lag 101   lag 100    lag 99  lag 98  ...  \
2020-03-18 08:15:00 -6.651707  2.325521  4.579251  5.320413  4.3501  ...   

                      lag 10     lag 9     lag 8     lag 7     lag 6  \
2020-03-18 08:15:00 -0.86537 -2.925028 -5.527036 -7.715383 -6.562472   

                        lag 5     lag 4     lag 3     lag 2     lag 1  
2020-03-18 08:15:00  2.003826  6.562201  8.005548  8.361737  8.229128  

[1 rows x 107 columns]
[8.229125]
8.57630044206707
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-18 08:30:00 -0.916628 -3.067816 -5.736217 -8.008443 -6.651707   

                      lag 102   lag 101   lag 100  lag 99    lag 98  ...  \
2020-03-18 08:30:00  2.325521  4.579251  5.320413  4.3501  3.840715  ...   

                       lag 10     lag 9     lag 8     lag 7

                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-18 11:30:00  4.077589  5.051811  6.046251  6.799981  8.385132   

                      lag 102   lag 101   lag 100   lag 99    lag 98  ...  \
2020-03-18 11:30:00  8.762086  8.881117  8.656705  9.22273  9.333564  ...   

                        lag 10      lag 9      lag 8      lag 7      lag 6  \
2020-03-18 11:30:00  10.668364  11.455154  11.364621  11.798406  12.010332   

                         lag 5      lag 4      lag 3      lag 2      lag 1  
2020-03-18 11:30:00  12.311329  12.185551  11.739007  11.663775  11.592368  

[1 rows x 107 columns]
[11.592365]
10.799102682437503
                      lag 107   lag 106   lag 105   lag 104   lag 103  \
2020-03-18 11:45:00  5.051811  6.046251  6.799981  8.385132  8.762086   

                      lag 102   lag 101  lag 100    lag 99     lag 98  ...  \
2020-03-18 11:45:00  8.881117  8.656705  9.22273  9.333564  10.589206  ...   

                        lag 

KeyboardInterrupt: 

In [ ]:
mse2 = runModels(df2,df1.iloc[-107:])

In [ ]:
mse3 = runModels(df3,df2.iloc[-107:])

In [ ]:
mse4 = runModels(df4,df3.iloc[-107:])

In [ ]:
print('season 1')

print('model 1:')
print(mse1)



In [ ]:
print('season 1')

print('model 1:')
print(mse2)


In [ ]:
print('season 1')

print('model 1:')
print(mse3)



In [ ]:
print('season 1')

print('model 1:')
print(mse4)



In [ ]:
d = {'Season 1':mse1,'Season 2':mse2,'Season 3':mse3,'Season 4':mse4}
df_errors_GBDT = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})

#df_errors_GBDT.to_csv('GBDT_errors.csv')
